<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/keras-example/15_optimization/036_colab_mnist_keras_example_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 2.1 A first look at a neural network

* from : F. Chollet, Deep Learning with Python, ISBN 9781617294433, 2017
* https://github.com/fchollet/deep-learning-with-python-notebooks
* https://www.manning.com/books/deep-learning-with-python
* Help from Gemini for the acceleration code.



In [ ]:
import os
import pathlib

import numpy as np
import pandas as pd
import tensorflow as tf



In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  print('not successful : tf.config.experimental.set_memory_growth()')
else:
  print('Using GPU')



### Listing 2.1 Loading the MNIST dataset in Keras



In [ ]:
try:
  data_folder = pathlib.Path('sample_data')
  assert data_folder.exists()
  assert data_folder.is_dir()

  def read_data(data_path):
    df = pd.read_csv(
        data_path,
        header=None
    )
    labels = np.array(df.iloc[:, 0])
    images = np.array(df.iloc[:, 1:])

    return images, labels


  train_images, train_labels = read_data(data_folder / 'mnist_train_small.csv')
  test_images, test_labels = read_data(data_folder / 'mnist_test.csv')

except AssertionError:
  (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()



### Listing 2.2 The training data


In [ ]:
train_images.shape



In [ ]:
n_train = len(train_labels)
n_train



In [ ]:
train_labels



### Listing 2.3 The test data


In [ ]:
test_images.shape



In [ ]:
n_test = len(test_labels)
n_test



In [ ]:
test_labels



### Listing 2.4 The network architecture



In [ ]:
network = tf.keras.models.Sequential()
network.add(tf.keras.layers.Input(shape=(28 * 28,)))  # Define input shape using Input layer
network.add(tf.keras.layers.Dense(512, activation='relu'))
network.add(tf.keras.layers.Dense(10, activation='softmax'))



### 2.5 The compilation step



In [ ]:
network.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



### 2.6 Preparing the image data



In [ ]:
train_images = np.array(train_images).reshape((n_train, 28 * 28))
train_images = np.array(train_images).astype('float32') / 255

test_images = np.array(test_images).reshape((n_test, 28 * 28))
test_images = np.array(test_images).astype('float32') / 255



### 2.7 Preparing the labels



In [ ]:
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)



### 2.8 Training the network



In [ ]:
n_epoch = 1 if os.getenv('GITHUB_ACTIONS', False) else 5



In [ ]:
%%time
network.fit(train_images, train_labels, epochs=n_epoch, batch_size=128)



### 2.9 Evaluating the network



In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

